In [3]:
import pandas as pd
import requests

In [8]:
import requests

url = "https://party.ect.go.th/party-info"
# Set verify=False to skip the SSL check
response = requests.get(url, verify=False)

# This hides the 'InsecureRequestWarning' that pops up when verify=False
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

response.encoding = response.apparent_encoding
html_text = response.text

c:\Users\ZhongFei.Ong\route_engineering\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'party.ect.go.th'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [67]:
from bs4 import BeautifulSoup
import requests
import urllib3
import json
import re

# Suppress SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "https://party.ect.go.th/party-info"
response = requests.get(url, verify=False)
response.raise_for_status()

soup = BeautifulSoup(response.content, "html.parser")


In [73]:
# Extract Thai and English party names from the HTML
import re
import json

# Get the full HTML text from the response
html_text = response.text

# Use regex to find the JSON array containing party info
match = re.search(r'"parties":(\[.*?\])', html_text, re.DOTALL)
if match:
    parties_json = match.group(1)
    # Clean up trailing commas or incomplete JSON if needed
    parties_json = parties_json.rstrip(',]')
    parties_json = parties_json + ']'
    try:
        parties = json.loads(parties_json)
        for party in parties:
            print(f"Thai: {party['name_th']}, English: {party['name_en']}")
    except Exception as e:
        print("Error parsing JSON:", e)
else:
    print("Party info not found in HTML.")

Party info not found in HTML.


In [74]:
html_text

'<!DOCTYPE html><html lang="th"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="stylesheet" href="/_next/static/chunks/fca7c5a484cde728.css" data-precedence="next"/><link rel="preload" as="script" fetchPriority="low" href="/_next/static/chunks/1d0ffb0701dea6db.js"/><script src="/_next/static/chunks/5da32ce39583d23f.js" async=""></script><script src="/_next/static/chunks/d1fa91ced486f727.js" async=""></script><script src="/_next/static/chunks/e5786566ed3572ef.js" async=""></script><script src="/_next/static/chunks/fe1a42ee2d00aff6.js" async=""></script><script src="/_next/static/chunks/8a576b3076bb1003.js" async=""></script><script src="/_next/static/chunks/turbopack-166b1eae20d9307b.js" async=""></script><script src="/_next/static/chunks/ff1a16fafef87110.js" async=""></script><script src="/_next/static/chunks/6af6f5d809ae5e33.js" async=""></script><script src="/_next/static/chunks/a0e026b365492527.js" async=""></script><scrip

In [88]:
import requests
import urllib3
from bs4 import BeautifulSoup
import re
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

url = "https://party.ect.go.th/party-info"
response = requests.get(url, verify=False)
response.raise_for_status()

soup = BeautifulSoup(response.content, "html.parser")

# Find all script tags
scripts = soup.find_all("script")

party_json = None
for script in scripts:
    if script.string and 'parties' in script.string:
        print('yes')
        # Extract the JSON part using regex
        # match = re.search(r'"parties":(\[.*?\])', script.string, re.DOTALL)
        # Use a non-greedy regex to extract the parties array
        match = re.search(r'"parties":(\[.*?\])', script.string, re.DOTALL)
        if match:
            parties_json = match.group(1)
            try:
                parties = json.loads(parties_json)
                for party in parties:
                    print(f"Thai: {party['name_th']}, English: {party['name_en']}")
            except Exception as e:
                print("Error parsing JSON:", e)
        else:
            print("Party info not found in script string.")
        break
        if match:
            parties_json = match.group(1)
            # Clean up trailing commas or incomplete JSON if needed
            parties_json = parties_json.rstrip(',]')
            parties_json = parties_json + ']'
            try:
                parties = json.loads(parties_json)
                for party in parties:
                    print(f"Thai: {party['name_th']}, English: {party['name_en']}")
            except Exception as e:
                print("Error parsing JSON:", e)
            break
else:
    print("Party info not found in HTML.")

yes
Party info not found in script string.


In [90]:
script.string

'self.__next_f.push([1,"6:[\\"$\\",\\"$8\\",null,{\\"fallback\\":[\\"$\\",\\"div\\",null,{\\"className\\":\\"fixed inset-0 flex items-center justify-center bg-white dark:bg-black accessible:bg-black z-50\\",\\"children\\":[\\"$\\",\\"div\\",null,{\\"className\\":\\"text-center\\",\\"children\\":[[\\"$\\",\\"svg\\",null,{\\"className\\":\\"animate-spin text-primary accessible:text-secondary mx-auto\\",\\"xmlns\\":\\"http://www.w3.org/2000/svg\\",\\"fill\\":\\"none\\",\\"viewBox\\":\\"0 0 24 24\\",\\"width\\":\\"40\\",\\"height\\":\\"40\\",\\"children\\":[[\\"$\\",\\"circle\\",null,{\\"className\\":\\"opacity-25\\",\\"cx\\":\\"12\\",\\"cy\\":\\"12\\",\\"r\\":\\"10\\",\\"stroke\\":\\"currentColor\\",\\"strokeWidth\\":\\"4\\"}],[\\"$\\",\\"path\\",null,{\\"className\\":\\"opacity-75\\",\\"fill\\":\\"currentColor\\",\\"d\\":\\"M4 12a8 8 0 018-8v4a4 4 0 00-4 4H4z\\"}]]}],[\\"$\\",\\"div\\",null,{\\"className\\":\\"mt-6 text-xl font-semibold text-primary accessible:text-secondary flex justify

In [98]:
import json
import re

raw_data = script.string

def extract_to_dataframe(data_string):
    # 1. Regex patterns for each field
    # These account for the backslash-escaped quotes found in your string
    codes = re.findall(r'\\"code\\":\\"(.*?)\\"', data_string)
    th_names = re.findall(r'\\"name_th\\":\\"(.*?)\\"', data_string)
    en_names = re.findall(r'\\"name_en\\":\\"(.*?)\\"', data_string)

    # 2. Combine into a list of dictionaries (handles potential length mismatches)
    data = []
    for c, th, en in zip(codes, th_names, en_names):
        data.append({
            "Party Code": c,
            "Thai Name": th,
            "English Name": en.replace("\\'", "'") # Fixes names like "Thailand 's Future"
        })

    # 3. Create DataFrame
    df = pd.DataFrame(data)
    return df

# Execute
df_parties = extract_to_dataframe(raw_data)

# Display result
display(df_parties)


#### save to excel
df_parties.to_excel("thai_political_parties.xlsx", index=False)

,Party Code,Thai Name,English Name
0,001,ประชาธิปัตย์,DEMOCRAT PARTY
1,002,ประชากรไทย,THAI CITIZEN PARTY
2,013,ความหวังใหม่,NEW ASPIRATION PARTY
3,022,เครือข่ายชาวนาแห่งประเทศไทย,THE FARMER NETWORK OF THAILAND PARTY
4,034,เพื่อไทย,PHEU THAI PARTY
...,...,...,...
70,254,ไทยทรัพย์ทวี,THAISUBTHAWEE PARTY
71,255,รวมพลังประชาชน,People's Power Party
72,256,อนาคตไกล,FAR FUTURE PARTY
73,257,ยางพาราไทย,Thai Rubber Party
